In [1]:
import numpy as np
import math
import modules
import optimization
from optimization import OpData
import copy
from scipy.optimize import minimize

max_iter = 2000


def ratio_finder(f):
    # Perturbation Size Δx Choice based on Gill et. al. 1983
    eps = 1e-6
    delta_X = math.sqrt(eps / abs(f))
    
    print('\nratio= ', "{:1.2f}".format(delta_X*100), '%')
    ratios = [1-delta_X/2, 1, 1+delta_X/2]
    return ratios

def sensitivity_sweep(v1_name, v2_name, all_vars):
    # v1: variable being swept, held constant for each optimization
    # v2: variable being optimized
    v2 = OpData(v2_name)
    v1 = OpData(v1_name)
    
    if ('wave_height' in v1.list):
        wave_period_i, wave_height_i = modules.load_wave_data("wave_data/Wave_Data2.csv")
        v1.nom_dict['wave_height'] = wave_height_i
        v1.nom_dict['wave_period'] = wave_period_i

    res_opt0, op_obj0, _ = optimization.run_optimization(v2.name, v2.nom0, v1.name, v1.nom_dict, all_vars, max_iter)

    v1_star_dict = copy.copy(v1.nom_dict)

    num_v1 = len(v1.list)
    num_v2 = len(v2.list)
    v1_ratios = ratio_finder(res_opt0.fun)
    num_ratios = len(v1_ratios)
    
    v2_opt = np.zeros(shape=(num_v1,num_ratios,num_v2))
    dv2_opt_dv1 = np.zeros(shape=(num_v1,num_v2))

    desc = '\nFinding optimal '+ v2.label + ' while holding '+ v1.label + ' as parameter:'
    print(desc)
    #print('ratio=', v1_ratios)
    
    for i in range(num_v1):
        for j in range(num_ratios):
            v1_vals = dict. copy(v1.nom_dict)
            v1_vals[v1.list[i]] = v1.nom_dict[v1.list[i]] * v1_ratios[j]
            #print(v1_vals)
            res_opt, op_obj, _ = optimization.run_optimization(v2.name, res_opt0.x, v1.name, v1_vals, all_vars, max_iter)
            if res_opt.success:
                v2_opt[i,j,:] = res_opt.x
            else:
                v2_opt[i,j,:] = np.nan
                print('failing for:',v1)
            #print('>>>>>',v2_opt[i,j,:])
            

        idx_nom = v1_ratios.index(1)
        dv2_opt = np.divide((v2_opt[i,-1,:] - v2_opt[i,0,:]), v2_opt[i,idx_nom,:])
        dv2_opt_dv1[i,:] = dv2_opt / (v1_ratios[-1] - v1_ratios[0])    

    # print result in a table format
    print('\n')
    col_width = len(max(v2.list+v1.list, key=len)) + 1
    print(' '*col_width, end='')
    for i in range(num_v2):
        print(' ',v2.list[i], ' '*(col_width-len(v2.list[i])-2),end='')
    print('')
    for i in range(num_v1):
        print(v1.list[i], ' '*(col_width-len(v1.list[i])),end='|')
        for j in range(num_v2):
            if (dv2_opt_dv1[i,j] == 0):
                print('  0', ' '*(col_width-4),end='|')
            elif np.isnan(dv2_opt_dv1[i,j]):
                print("  \x1b[31mFailed\x1b[0m", ' '*(col_width-9),end='|')
            else:
                print("{:10.6f}".format(dv2_opt_dv1[i,j]), ' '*(col_width-11),end='|')
        print('')
        
    return dv2_opt_dv1


all_vars = ['x_wec','x_type_wec','x_pen', 'x_disc_pen', 'p_pen','x_env','p_wec','p_fish_salmon', 'x_es', 'p_es', 'pos_env', 'p_vessel']

# sensitivity of optimal wec and pen design to environment/location with wave real data
design_var = ['x_wec','x_pen', 'x_es']
parameter = ['x_env','p_wave_vec']
dx_wec_pen_star_dx_env = sensitivity_sweep(parameter, design_var, all_vars + ['p_wave_vec'])
#print('\n\ndx_wec_pen_star_dx_env=\n', dx_wec_pen_star_dx_env)
#print('\n*************************************************\n\n')

# sensitivity of optimal environment/location to wec and pen design with wave average data
design_var = ['x_env', 'x_wave_ave']
parameter = ['x_wec','x_pen', 'x_es']
dx_env_star_dx_wec_pen = sensitivity_sweep(parameter, design_var, all_vars + ['x_wave_ave'])
#print('\n\ndx_env_star_dx_wec_pen=\n', dx_env_star_dx_wec_pen)
#print('\n*************************************************\n\n')


ratio=  0.03 %

Finding optimal x_wec & x_pen & x_es while holding x_env & p_wave_vec as parameter:


                capture_width   pen_diameter   pen_height     spacing        stock_density   pen_depth      es_size      
temp           |  0.000000    |  0           |  0.000000    |  0           |  0           |  0           |  0           |
O2_in          |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
salinity       |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
U              |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
wave_height    | -2.001124    | -0.000044    | -0.001033    |  0           |  0           |  0           | -0.001120    |
wave_period    | -1.000210    | -0.000181    |  0.000296    |  0           |  0           |  0           |  0.000198    |

ratio=  0.02 %

Finding optimal x_env & x_